In [1]:
from acdesign.performance.aero import WingAero
from acdesign.airfoils.polar import UIUCPolar, uiuc_airfoils
import numpy as np

wings = dict(
    clarky=WingAero(
        b=2, 
        S=2 * 0.1, 
        polars=[UIUCPolar.local("CLARKYB")], rib_locs=[0, 1]
    ),
    rg15=WingAero(
        b=2,
        S=2 * 0.1,
        polars=[UIUCPolar.local("RG15C")],
        rib_locs=[0, 1],
#        spanchord=lambda yb: np.sqrt(1 - yb**2),
    ),
)

In [15]:
from acdesign.atmosphere import Atmosphere
import plotly.express as px
import plotly.graph_objects as go

atm = Atmosphere.alt(0)

cls = np.linspace(0, 1, 20)
vs = np.linspace(10, 15, 3)

results = {k: v(atm=atm, v=vs, cl=cls).reset_index() for k, v in wings.items()}

fig = go.Figure()

# print(results)
for iw, (k, result) in enumerate(results.items()):
    for i, v in enumerate(vs):
        df = result.loc[result.fs_v == v]
        fig.add_trace(
            go.Scatter(
                x=df.lift,
                y=df.drag,
                mode="lines",
                name=f"{k}, V={v:.1f} m/s",
                line=dict(
                    dash=["solid", "dash", "dotted"][iw],
                    color=px.colors.qualitative.Plotly[i],
                ),
            )
        )


fig.update_layout(
    template="plotly_white",
    title="Example Wing Lift vs Drag",
    xaxis_title="Lift (N)",
    yaxis_title="Drag (N)",
    margin=dict(l=0, r=0, t=40, b=0),
    width=800, height=400,
)